# Modularizing DHSAlumni Data

In [1]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [42]:
#Custom Owner Metadata Dictionary

owners_dict = {"owner_id": [1, 2, 3, 5, 6, 7, 8, 9, 11, 12],
               "owner_name": ["Zwick", "Treshansky", "Lirtzman", "Klein", "Rosenberg", "Katz", "J.Stein", "Berkowitz", 
                              "Kogan", "B.Stein/Kaminski"]}

owners_df = pd.DataFrame(owners_dict)
owners_df

,owner_id,owner_name
0,1,Zwick
1,2,Treshansky
2,3,Lirtzman
3,5,Klein
4,6,Rosenberg
5,7,Katz
6,8,J.Stein
7,9,Berkowitz
8,11,Kogan
9,12,B.Stein/Kaminski


In [10]:
#Additional Parameters for Pinging API

league_id = 470074
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018]
#url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/" + \
      #str(league_id) + "?seasonId=" + str(year)

In [34]:
#Empty Matchup Dataframe
matchups = pd.DataFrame()

In [40]:
for year in years:
    
    #Setting URL and cookie parameters
    url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/" + str(league_id) + "?seasonId=" + str(year)
    swid = "{0E55E5A5-3637-4BE0-86CF-E2E39B9938C9}"
    espn_s2 = "AEAZNuutax36J%2BmmE9X0oeTTRyhq0DrK2gcqVGB7Cxc75XXs4TJxz73ekvnJdz33IFF4fJXslPhu" + \
          "n7SEmT9Q4c%2Fv1jXfA%2B4Kl6scPVRUks%2B5xLpe4XvomcuqovBxd6ZoSDpzoMJ%2FpjoUsuzVf" + \
          "lGKkdgv6ZlBsl%2Bc7s3VgBprIHx6juy5SV9ac3PVT%2F5Su%2FAb51g7p60Ebxw6Qi%2FaGj%2B7Y8sq4sPlNsqW32UXk2xJJH" + \
          "%2BP5%2B14PFH2ky%2BHeNwRewSBDMa3zsdLPlWnw9kbm6GLdvesu"
    
    #Hitting API and getting Matchup View
    r = requests.get(url, cookies={"swid": swid, "espn_s2": espn_s2}, params={"view": "mMatchup"})
    m = r.json()[0]
    
    game_lst = [[game['matchupPeriodId'], game['home']['teamId'], game['home']['totalPoints'],
                game['away']['teamId'], game['away']['totalPoints'], year] for game in m['schedule']]
    
    matchups = matchups.append(game_lst, ignore_index=True)

In [47]:
matchups.columns = ["Week", "HomeTeam", "HomeScore", "AwayTeam", "AwayScore", "Year"]
matchups["Type"] = ["Regular" if w <=14 else "Playoff" for w in matchups["Week"]]
matchups["AwayTeamOwner"] = matchups.AwayTeam.map(owners_df.owner_name)

ValueError: Length mismatch: Expected axis has 7 elements, new values have 6 elements

In [46]:
matchups.head()

,Week,HomeTeam,HomeScore,AwayTeam,AwayScore,Year,Type
0,1,2,143.40,1,146.60,2012,Regular
1,1,12,146.46,3,107.60,2012,Regular
2,1,11,118.46,5,140.56,2012,Regular
3,1,9,139.98,6,128.00,2012,Regular
4,1,8,126.36,7,131.32,2012,Regular


In [51]:
matchups = matchups.drop(["AwayTeamOwner"], axis=1)

In [59]:
matchups.head()

,Week,HomeTeam,HomeScore,AwayTeam,AwayScore,Year,Type
0,1,2,143.40,1,146.60,2012,Regular
1,1,12,146.46,3,107.60,2012,Regular
2,1,11,118.46,5,140.56,2012,Regular
3,1,9,139.98,6,128.00,2012,Regular
4,1,8,126.36,7,131.32,2012,Regular


In [60]:
owners_df

,owner_id,owner_name
0,1,Zwick
1,2,Treshansky
2,3,Lirtzman
3,5,Klein
4,6,Rosenberg
5,7,Katz
6,8,J.Stein
7,9,Berkowitz
8,11,Kogan
9,12,B.Stein/Kaminski


In [64]:
merged = pd.merge(matchups, owners_df, how="left", left_on="AwayTeam", right_on="owner_id")
merged.head()

,Week,HomeTeam,HomeScore,AwayTeam,AwayScore,Year,Type,owner_id,owner_name
0,1,2,143.40,1,146.60,2012,Regular,1,Zwick
1,1,12,146.46,3,107.60,2012,Regular,3,Lirtzman
2,1,11,118.46,5,140.56,2012,Regular,5,Klein
3,1,9,139.98,6,128.00,2012,Regular,6,Rosenberg
4,1,8,126.36,7,131.32,2012,Regular,7,Katz


In [66]:
merged = merged.drop(["owner_id"], axis=1)

KeyError: "['owner_id'] not found in axis"

In [67]:
merged.head()

,Week,HomeTeam,HomeScore,AwayTeam,AwayScore,Year,Type,owner_name
0,1,2,143.40,1,146.60,2012,Regular,Zwick
1,1,12,146.46,3,107.60,2012,Regular,Lirtzman
2,1,11,118.46,5,140.56,2012,Regular,Klein
3,1,9,139.98,6,128.00,2012,Regular,Rosenberg
4,1,8,126.36,7,131.32,2012,Regular,Katz


In [68]:
merged = merged.rename(columns={"owner_name": "AwayTeamName"})
merged.head()

,Week,HomeTeam,HomeScore,AwayTeam,AwayScore,Year,Type,AwayTeamName
0,1,2,143.40,1,146.60,2012,Regular,Zwick
1,1,12,146.46,3,107.60,2012,Regular,Lirtzman
2,1,11,118.46,5,140.56,2012,Regular,Klein
3,1,9,139.98,6,128.00,2012,Regular,Rosenberg
4,1,8,126.36,7,131.32,2012,Regular,Katz


In [70]:
merged2 = pd.merge(merged, owners_df, how="left", left_on="HomeTeam", right_on="owner_id")
merged2.head()

,Week,HomeTeam,HomeScore,AwayTeam,AwayScore,Year,Type,AwayTeamName,owner_id,owner_name
0,1,2,143.40,1,146.60,2012,Regular,Zwick,2,Treshansky
1,1,12,146.46,3,107.60,2012,Regular,Lirtzman,12,B.Stein/Kaminski
2,1,11,118.46,5,140.56,2012,Regular,Klein,11,Kogan
3,1,9,139.98,6,128.00,2012,Regular,Rosenberg,9,Berkowitz
4,1,8,126.36,7,131.32,2012,Regular,Katz,8,J.Stein


In [71]:
merged2 = merged2.drop(["owner_id"], axis=1)

In [72]:
merged2 = merged2.rename(columns={"owner_name": "HomeTeamName"})

In [73]:
merged2.head()

,Week,HomeTeam,HomeScore,AwayTeam,AwayScore,Year,Type,AwayTeamName,HomeTeamName
0,1,2,143.40,1,146.60,2012,Regular,Zwick,Treshansky
1,1,12,146.46,3,107.60,2012,Regular,Lirtzman,B.Stein/Kaminski
2,1,11,118.46,5,140.56,2012,Regular,Klein,Kogan
3,1,9,139.98,6,128.00,2012,Regular,Rosenberg,Berkowitz
4,1,8,126.36,7,131.32,2012,Regular,Katz,J.Stein


In [75]:
merged2["Winner"] = np.where(merged2["HomeScore"] > merged2["AwayScore"], merged2["HomeTeamName"], merged2["AwayTeamName"])
merged2["Loser"] = np.where(merged2["HomeScore"] > merged2["AwayScore"], merged2["AwayTeamName"], merged2["HomeTeamName"])

In [76]:
merged2

,Week,HomeTeam,HomeScore,AwayTeam,AwayScore,Year,Type,AwayTeamName,HomeTeamName,Winner,Loser
0,1,2,143.40,1,146.60,2012,Regular,Zwick,Treshansky,Zwick,Treshansky
1,1,12,146.46,3,107.60,2012,Regular,Lirtzman,B.Stein/Kaminski,B.Stein/Kaminski,Lirtzman
2,1,11,118.46,5,140.56,2012,Regular,Klein,Kogan,Klein,Kogan
3,1,9,139.98,6,128.00,2012,Regular,Rosenberg,Berkowitz,Berkowitz,Rosenberg
4,1,8,126.36,7,131.32,2012,Regular,Katz,J.Stein,Katz,J.Stein
5,2,1,137.74,3,113.66,2012,Regular,Lirtzman,Zwick,Zwick,Lirtzman
6,2,5,128.36,2,126.10,2012,Regular,Treshansky,Klein,Klein,Treshansky
7,2,6,180.44,12,125.86,2012,Regular,B.Stein/Kaminski,Rosenberg,Rosenberg,B.Stein/Kaminski
8,2,7,173.50,11,124.94,2012,Regular,Kogan,Katz,Katz,Kogan
9,2,8,86.88,9,123.92,2012,Regular,Berkowitz,J.Stein,Berkowitz,J.Stein


In [77]:
regular_season = merged2[merged2["Type"] == "Regular"]
regular_season

,Week,HomeTeam,HomeScore,AwayTeam,AwayScore,Year,Type,AwayTeamName,HomeTeamName,Winner,Loser
0,1,2,143.40,1,146.60,2012,Regular,Zwick,Treshansky,Zwick,Treshansky
1,1,12,146.46,3,107.60,2012,Regular,Lirtzman,B.Stein/Kaminski,B.Stein/Kaminski,Lirtzman
2,1,11,118.46,5,140.56,2012,Regular,Klein,Kogan,Klein,Kogan
3,1,9,139.98,6,128.00,2012,Regular,Rosenberg,Berkowitz,Berkowitz,Rosenberg
4,1,8,126.36,7,131.32,2012,Regular,Katz,J.Stein,Katz,J.Stein
5,2,1,137.74,3,113.66,2012,Regular,Lirtzman,Zwick,Zwick,Lirtzman
6,2,5,128.36,2,126.10,2012,Regular,Treshansky,Klein,Klein,Treshansky
7,2,6,180.44,12,125.86,2012,Regular,B.Stein/Kaminski,Rosenberg,Rosenberg,B.Stein/Kaminski
8,2,7,173.50,11,124.94,2012,Regular,Kogan,Katz,Katz,Kogan
9,2,8,86.88,9,123.92,2012,Regular,Berkowitz,J.Stein,Berkowitz,J.Stein


In [80]:
regular_season["Winner"].value_counts()

Zwick               96
Katz                90
J.Stein             89
Berkowitz           86
Klein               86
Lirtzman            73
B.Stein/Kaminski    73
Treshansky          70
Rosenberg           63
Kogan               44
Name: Winner, dtype: int64

In [81]:
regular_season["Loser"].value_counts()

Kogan               110
Rosenberg            91
Treshansky           84
Lirtzman             81
B.Stein/Kaminski     81
Berkowitz            68
Klein                68
J.Stein              65
Katz                 64
Zwick                58
Name: Loser, dtype: int64

In [85]:
totals = pd.DataFrame(columns=["Wins", "Losses"])
totals

,Wins,Losses


In [86]:
totals["Wins"] = regular_season["Winner"].value_counts()
totals

,Wins,Losses
Zwick,96,NaN
Katz,90,NaN
J.Stein,89,NaN
Berkowitz,86,NaN
Klein,86,NaN
Lirtzman,73,NaN
B.Stein/Kaminski,73,NaN
Treshansky,70,NaN
Rosenberg,63,NaN
Kogan,44,NaN


In [87]:
totals["Losses"] = regular_season["Loser"].value_counts()
totals

,Wins,Losses
Zwick,96,58
Katz,90,64
J.Stein,89,65
Berkowitz,86,68
Klein,86,68
Lirtzman,73,81
B.Stein/Kaminski,73,81
Treshansky,70,84
Rosenberg,63,91
Kogan,44,110


## Total Regular Season Wins and Losses 2012-2018

In [88]:
totals

,Wins,Losses
Zwick,96,58
Katz,90,64
J.Stein,89,65
Berkowitz,86,68
Klein,86,68
Lirtzman,73,81
B.Stein/Kaminski,73,81
Treshansky,70,84
Rosenberg,63,91
Kogan,44,110


In [92]:
regular_season.groupby(["Winner", "Loser"]).size()

Winner            Loser           
B.Stein/Kaminski  Berkowitz            4
                  J.Stein              9
                  Katz                 9
                  Klein                7
                  Kogan               17
                  Lirtzman             9
                  Rosenberg            7
                  Treshansky           5
                  Zwick                6
Berkowitz         B.Stein/Kaminski     7
                  J.Stein              5
                  Katz                 7
                  Klein               11
                  Kogan               15
                  Lirtzman             9
                  Rosenberg           13
                  Treshansky          15
                  Zwick                4
J.Stein           B.Stein/Kaminski    11
                  Berkowitz           13
                  Katz                 9
                  Klein                9
                  Kogan                9
                  Lirt

In [96]:
berk = regular_season[(regular_season["HomeTeamName"] == "Berkowitz") | (regular_season["AwayTeamName"] == "Berkowitz")]

In [97]:
berk

,Week,HomeTeam,HomeScore,AwayTeam,AwayScore,Year,Type,AwayTeamName,HomeTeamName,Winner,Loser
3,1,9,139.98,6,128.00,2012,Regular,Rosenberg,Berkowitz,Berkowitz,Rosenberg
9,2,8,86.88,9,123.92,2012,Regular,Berkowitz,J.Stein,Berkowitz,J.Stein
14,3,11,115.88,9,122.34,2012,Regular,Berkowitz,Kogan,Berkowitz,Kogan
18,4,9,90.52,2,130.52,2012,Regular,Treshansky,Berkowitz,Treshansky,Berkowitz
22,5,5,170.54,9,144.68,2012,Regular,Berkowitz,Klein,Klein,Berkowitz
26,6,9,152.92,7,123.98,2012,Regular,Katz,Berkowitz,Berkowitz,Katz
30,7,9,133.12,1,139.04,2012,Regular,Zwick,Berkowitz,Zwick,Berkowitz
36,8,12,162.96,9,142.12,2012,Regular,Berkowitz,B.Stein/Kaminski,B.Stein/Kaminski,Berkowitz
42,9,9,191.94,3,133.16,2012,Regular,Lirtzman,Berkowitz,Berkowitz,Lirtzman
48,10,6,168.58,9,156.46,2012,Regular,Berkowitz,Rosenberg,Rosenberg,Berkowitz


In [98]:
berk.groupby(["Winner", "Loser"]).size()

Winner            Loser           
B.Stein/Kaminski  Berkowitz            4
Berkowitz         B.Stein/Kaminski     7
                  J.Stein              5
                  Katz                 7
                  Klein               11
                  Kogan               15
                  Lirtzman             9
                  Rosenberg           13
                  Treshansky          15
                  Zwick                4
J.Stein           Berkowitz           13
Katz              Berkowitz            8
Klein             Berkowitz            9
Kogan             Berkowitz            6
Lirtzman          Berkowitz            7
Rosenberg         Berkowitz            3
Treshansky        Berkowitz            5
Zwick             Berkowitz           13
dtype: int64